In [ ]:
''' 
Algorithm Description:

This Algorithm buys n = 40  number of stocks on a random basis and holds them for 252 days. After the initial 252 days the
algorithm sells all holdings and pick another random n stocks to invest. No transactions costs and equal weights. Weights 
are not adjusted during the holding period.
'''
#Importing
import quantopian.algorithm as algo
from quantopian.pipeline import Pipeline
from quantopian.algorithm import attach_pipeline, pipeline_output
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.data import Fundamentals
import random

random.seed(12) #randomly generated seed by picking a random number between 1 and 100

def initialize(context):
    """
    Called once at the start of the algorithm.
    """
    
    #Function buy_random will be run every day, 1 hour after market open
    schedule_function(buy_random, 
                      date_rules.every_day(),
                      time_rules.market_open(hours = 1)) 
    
    context.first_buy = True #used in buy_random function to let it run
    
    #Function check_duration_and_sell is run every da as well
    schedule_function(check_duration_and_sell,
                      date_rules.every_day(),
                      time_rule = time_rules.market_open()) 
    
    #Function check_duration_and_buy is run every day but 1 hour after market open so that the sell function 
    #sells the positions first before buying
    schedule_function(check_duration_and_buy,
                      date_rules.every_day(),
                      time_rules.market_open(hours = 1)) 
    
    
    context.holding_time = {} #empty dictionary
    context.can_buy = False #This stops can_buy from running immediately
    
                           
    # Creates the pipeline used for selecting our universe
    algo.attach_pipeline(make_pipeline(), 'pipeline')


def make_pipeline():
    """
    Picks all stocks from the universe
    """

    universe = QTradableStocksUS() #Investment Universe is the American Stockmarket
    market_cap = Fundamentals.market_cap.latest #market_cap as example, could be anything as its only used to get all stocks

    pipe = Pipeline(
        columns={
            'stocks': market_cap, #taking mcap just so that we have a pipeline output
        },
        screen=universe #screen reduces calculation time
    )
    return pipe

def buy_random(context, data): 
    """
    Check what month it is and call corresponding function.
    Everything below the 'if' statement triggers only if first_buy = True, as set in 'initialize' function.
    n indicates the number of stocks to buy. This function only runs once at the start of the time period.
    """
    n = 40 #number of stocks that we want to randomly buy

    if context.first_buy == True:
        
        stocks = pipeline_output('pipeline').index.tolist() #take all stocks from pipeline output and put into list with name stocks
        random.shuffle(stocks) #randomly shuffle stocks
        context.stocks = [] #create empty list
        stockcount = 0
        for stock in stocks:
            context.stocks.append(stock)
            stockcount += 1
            if stockcount == n: #we only want to buy this number of stocks
                break
    
        for stock in context.stocks:
            order_target_percent(stock, 1.0 /len(context.stocks)) #give stocks equal weight
            context.holding_time[stock] = 0 #we give each stock a timer which tells us for how many days we have been holding it. 
                                            #Initial timer is set to zero.
        
        #This stopps this function from running forever. Therefore it only runs once in the beginning
        context.first_buy = False 
                                         
            
def check_duration_and_sell(context, data):
    """
    Checks if stocks were hold for 252 days, and if so sell them.
    """
    
    for stock in context.holding_time.keys(): # iterate through all stocks in the holding time dictionary  
        context.holding_time[stock] = context.holding_time[stock] + 1 # increment all stocks by 1 day
        

    for stock in context.portfolio.positions: # iterate through all stocks in the portfolio  
        if context.holding_time[stock] >= 252: # check if stock has hit 252 days  
            order_target_percent(stock, 0) # liquidate the stock
            context.can_buy = True #Sets can_buy to True so that the check_duration_and_buy function can start running. 
                                    #This is done so that buy and sell orders do not run at the same time.

def check_duration_and_buy(context, data):
    """
    Buys stocks and gives them equal weights.
    """

    if context.can_buy == True:
        n = 40 #amount of stocks we want to buy
        stocks = pipeline_output('pipeline').index.tolist() #take all stocks from pipeline output and put into list with name stocks
        random.shuffle(stocks) #randomly shuffle stocks
        context.stocks = [] #create empty list
        stockcount = 0
        for stock in stocks:
            context.stocks.append(stock)
            stockcount += 1
            context.holding_time[stock] = 0
            if stockcount == n: #we only want to buy this number of stocks
                break
         
    
        for stock in context.stocks:
            order_target_percent(stock, 1.0 /len(context.stocks)) #give stocks equal weight
    context.can_buy = False